## Running DeepSeek via Azure Foundry ##
- This specific notebook is to demonstrate the uses of langchain + deepseek and integration of Exa.AI as the backend search functionality

In [ ]:
# Import required packages
%pip install -r requirements.txt

## Putting the code together ##
This code block does the following
- Initializes our Azure AI Chat Completions Client via langchain_azure_ai.chat_models
- Initialize Exa and wraps it in the exa_search_function (takes query, k=3) to limit to 3 results.
- Defines the prompt template which is then 'chained' in the following code-block.

In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from dotenv import load_dotenv
from exa_py import Exa
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Get the environment variables
endpoint = os.getenv("AZURE_AI_ENDPOINT") # Azure AI endpoint environment variable this is the Completions Endpoint
model_name = os.getenv("AZURE_DEPLOYMENT") # Azure AI model name environment variable
key = os.getenv('AZURE_AI_KEY') # Azure AI key environment variable
exa_api = os.getenv('EXA_API_KEY') # Exa API key environment variable
region = os.getenv("REGION")   # Region the model is deployed in

wrapper_key = AzureKeyCredential(key) # Method of wrapping the key in AzureKeyCredential

ds_endpoint = f"https://{model_name}.{region}.models.ai.azure.com"

# Initialize the Azure AI Chat Completions Model
model = AzureAIChatCompletionsModel(
    endpoint=ds_endpoint,
    credential=AzureKeyCredential(key),
    model_name=model_name,
    max_tokens=2048
)

# Initialize the exa client
exa = Exa(exa_api)

def exa_search_function(query, k=3):
    """
    Function to search for the top k results from the Exa API and format them as relevant_info

    Returns:
       tuple: (revelant_info, error_message)
           - relevant_info: formatted string of search results or empty string if error
           - error_message: error message if any or None if successful
    """
    try:
        result = exa.search_and_contents(
            query=query,
            num_results=k,
            subpages=k,
            text=True
        )
        if not result or not hasattr(result, 'results') or not result.results:
            return "", "No search results found"
        # Format the results into a string
        relevant_info = ""
        for item in result.results:
            relevant_info += f"\nTitle: {item.title if hasattr(item, 'title') else 'Unknown'}\n"
            relevant_info += f"URL: {item.url if hasattr(item, 'url') else 'Unknown'}\n"
            # Check if the content attribute exists before trying to access it
            if hasattr(item, 'text') and item.text:
                text_except = item.text[:1000] + "..." if len(item.text) > 1000 else item.text
                relevant_info += f"Content: {text_except}\n"
            # Add summary if available
            if hasattr(item, 'summary') and item.summary:
                relevant_info += f"Summary: {item.summary}\n"
            relevant_info += "\n"
            
            # Process subpages if available
            if hasattr(item, 'subpages') and item.subpages:
                relevant_info += "Related pages:\n"
                for subpage in item.subpages[:2]:  # Limit to 2 subpages
                    relevant_info += f"  - {subpage.title if hasattr(subpage, 'title') else 'Unknown'}\n"
                    if hasattr(subpage, 'summary') and subpage.summary:
                        summary_excerpt = subpage.summary[:200] + "..." if len(subpage.summary) > 200 else subpage.summary
                        relevant_info += f"    Summary: {summary_excerpt}\n"
                relevant_info += "\n"
            
            
        return relevant_info, None
    except Exception as e:
        return "", str(e)
# Prompt template
prompt_template = PromptTemplate(
    input_variables = ["query", "relevant_info"],
    template = """
    You are an expert in Research of a variety of topics specifically AI Security. Use the following information from Exa Search to answer the user's question. If there is no sufficient information, say 'I need more information to answer this question'.

    Question: {query}

    Relevant Information:
    {relevant_info}

    Answer:
    """
)

# Added our chain #
This puts our prompt_template from the previous code + attaching the model.

In [10]:
# Defining the chain
chain = prompt_template | model

## Define our query ##

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="What is AI Security?")])

In [11]:
query = "What are the best methods to defend against Excessive Agency in AI Agents?"

# Check if API Key is set
if not exa_api:
    print("Please set the EXA_API_KEY environment variable to use the Exa API")

try:
    # Get search results using your existing function
    relevant_info, error = exa_search_function(query)
    
    if error:
        print(f"Error during search: {error}")
    else:
        # Process with LangChain
        response = chain.invoke({"query": query, "relevant_info": relevant_info})
        
        # Format the response
        response_text = response.content if hasattr(response, 'content') else response
        
        # Clear formatting for better display
        if isinstance(response_text, str):
            # Just use the response as is
            formatted_response = response_text
        else:
            # Handle AIMessage or other objects
            formatted_response = str(response_text)
            
            # Clean up the formatting - remove content= and quotes if present
            if formatted_response.startswith("content='"):
                formatted_response = formatted_response.split("content='", 1)[1].rsplit("'", 1)[0]
            
            # Remove any <think> sections if present
            if "<think>" in formatted_response and "</think>" in formatted_response:
                formatted_response = formatted_response.split("</think>", 1)[1].strip()

        # Take the response_text and format
        print("\n" + "=" * 80)
        print("\nQuery from User:")
        print("-" * 80)
        print(query)
        
        print("\n\nResponse:")
        print("-" * 80)
        print(formatted_response)
        
        print("\n\nExa Sources:")
        print("-" * 80)
        
        # Format relevant_info - extract just the titles and URLs for cleaner display
        import re
        sources = []
        for match in re.finditer(r'Title: (.*?)\nURL: (.*?)(?:\n|$)', relevant_info):
            title, url = match.groups()
            sources.append(f"• {title}\n  {url}")
        
        if sources:
            print("\n".join(sources))
            print("\n" + "-" * 80)
            print("\nFull source information:")
            print(relevant_info)
        else:
            print(relevant_info)
            
        print("=" * 80 + "\n")
        
except Exception as e:
    print(f"Error processing query: {e}")
    import traceback
    traceback.print_exc()



Query from User:
--------------------------------------------------------------------------------
What are the best methods to defend against Excessive Agency in AI Agents?


Response:
--------------------------------------------------------------------------------
<think>
Okay, so I need to figure out the best methods to defend against Excessive Agency in AI Agents based on the provided information. Let me go through each of the relevant sources to see what they suggest.

First, the Approval-directed agents article mentions an alternative to goal-directed behavior. It talks about how most AI safety research focuses on aligning the goals of AI with humans, but this piece suggests a different approach. Maybe instead of having autonomous agents with their own goals, approval-directed agents would seek human approval for their actions. That could prevent excessive agency by making the AI check in with humans before acting. But I need to see if that's explicitly stated. The content snipp